In [ ]:
!pip install langsmith > /dev/null
!pip install langchain > /dev/null
!pip install openai > /dev/null
!pip install python-dotenv > /dev/null

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("How are you?")

### Using Langsmith

In [ ]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="My Project")
llm.predict("How many people live in USA?", callbacks=[tracer])

In [ ]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled(project_name="My Project"):
    llm.predict("How many people live in Germany?")

In [ ]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0, tags=["my-llm-tag"])
prompt = PromptTemplate.from_template("Say {input}")
chain = LLMChain(llm=llm, prompt=prompt, tags=["one-tag", "another-tag"])

chain("Hello, World!", tags=["shared-tags"])

In [ ]:
from langchain.callbacks.manager import (
    trace_as_chain_group
)

with trace_as_chain_group("my_group_name") as group_manager:
    pass

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["question"],
    template="What is the answer to {question}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
with trace_as_chain_group("my_group") as group_manager:
    chain.run(question="What is your name?", callbacks=group_manager)
    chain.run(question="What is your quest?", callbacks=group_manager)
    chain.run(question="What is your favorite color?", callbacks=group_manager)

In [ ]:
from langsmith import Client

client = Client()
project_runs = client.list_runs(project_name="default")
project_runs

In [ ]:
from datetime import datetime, timedelta

todays_runs = client.list_runs(
    project_name="default",
    start_time=datetime.now() - timedelta(days=1),
    run_type="llm",
)
todays_runs


In [ ]:
for run in todays_runs:
    print(run)

### Metadata
Add metadata to filter runs later, for example for making an A/B test


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

chat_model = ChatOpenAI()
chain = LLMChain.from_string(llm=chat_model, template="What's the answer to {input}?")

chain({"input": "What is the meaning of life?"}, metadata={"source": "korean"})

In [ ]:
from langsmith import Client

client = Client()
runs = list(client.list_runs(
    project_name="default",
    filter='has(metadata, \'{"source": "kkrean"}\')',
))
print(list(runs))

### Datasets

In [ ]:
from langsmith import Client
client = Client()

dataset = client.create_dataset(dataset_name="testdataset", description="A dataset with key-value inputs and outputs")

In [ ]:
client.create_example(
  inputs={
    "a-question": "What is the largest mammal?",
    "user-context": "The user is a 1st grader writing a bio report.",
  },
  outputs = {
    "answer": "The blue whale is the largest mammal.",
    "source": "https://en.wikipedia.org/wiki/Blue_whale",
  },
  dataset_id=dataset.id,
)

In [ ]:
dataset = client.upload_csv(
    csv_file="./extended_questions_answers.csv",
    input_keys=["Question"],
    output_keys=["Answer"],
    name="My CSV Dataset",
    description="Dataset created from a CSV file",
    data_type="kv",
)

Evaluation and Datasets

In [ ]:
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    evaluators=[
        "qa",
        "context_qa",
        "cot_qa",
    ]
)

In [ ]:
client = Client()
llm = ChatOpenAI()
run_on_dataset(
    dataset_name="My CSV Dataset",
    llm_or_chain_factory=llm,
    client=client,
    evaluation=evaluation_config,
    project_name="evalproject",
)